In [1]:
import os, deepreload, warnings

os.environ["CDR_AUTH_TOKEN"] = (
    ""
)

warnings.filterwarnings("ignore")

In [2]:
from minmodkg.integrations.cdr import *

deepreload.reload("minmodkg.integrations.cdr")
from minmodkg.integrations.cdr import *

In [3]:
commodity = "nickel"

In [4]:
sites = get_mineral_site_data_by_commodity(commodity)

2024-07-15 23:05:29.243 | INFO     | minmodkg.integrations.cdr:get_mineral_site_data_by_commodity:116 - Having total of 4105 sites
retrieving additional information of mineral sites: 100%|██████████| 42/42 [00:02<00:00, 17.15it/s]
reformat mineral sites: 100%|██████████| 4105/4105 [00:00<00:00, 50289.21it/s]


In [5]:
create_mineral_site(sites)

creating mineral sites: 100%|██████████| 4105/4105 [31:32<00:00,  2.17it/s]  


In [16]:
r = httpx.post(
    f"{CDR_API}/minerals/site",
    json=orjson.loads(sites[0].model_dump_json(exclude_none=True)),
    headers=cdr_headers,
)

In [26]:
r = httpx.delete(
    f"{CDR_API}/minerals/site/{sites[0].id}",
    headers=cdr_headers,
)

In [29]:
r.status_code

404

In [23]:
sites[0].id

'site__10-1016-j-oregeorev-2016-08-010__23__area-55'

In [25]:
sites[0].model_dump_json(exclude_none=True)

'{"id":"site__10-1016-j-oregeorev-2016-08-010__23__area-55","source_id":"https://doi.org/10.1016/j.oregeorev.2016.08.010","record_id":"https://minmod.isi.edu/resource/site__10-1016-j-oregeorev-2016-08-010__23__area-55","name":"Area 55","site_rank":"","site_type":"","country":"Australia","province":[],"mineral_inventory":[{"contained_metal":0.017079999999999998,"commodity":"nickel","commodity_observed_name":"","ore_unit":"","ore_value":12.2,"grade_unit":"","grade_value":0.13999999999999999,"cutoff_grade_unit":"","material_form_unit":"","categories":[],"documents":[],"records":[],"date":"","zone":""}],"deposit_type_candidate":[],"validated":false,"system":"minmodkg","system_version":"0.1.0"}'

In [3]:
commodity = "nickel"
deposity_type_resp = httpx.get(
    f"{MINMOD_API}/mineral_site_deposit_types/{commodity}",
    verify=False,
    timeout=None,
)
deposity_type_resp.raise_for_status()

grade_tonnage_resp = httpx.get(
    f"{MINMOD_API}/mineral_site_grade_and_tonnage/{commodity}",
    verify=False,
    timeout=None,
)
grade_tonnage_resp.raise_for_status()

<Response [200 OK]>

In [4]:
site_ids = list(
    {record["ms"][len(MNR_NS) :] for record in deposity_type_resp.json()}.union(
        (record["ms"][len(MNR_NS) :] for record in grade_tonnage_resp.json())
    )
)

In [5]:
logger.info("Having total of {} sites", len(site_ids))

2024-07-15 22:27:16.499 | INFO     | __main__:<module>:1 - Having total of 4105 sites


In [10]:
id2siteinfo = {}
for batch_ids in tqdm(
    batch(100, site_ids), desc="retrieving additional information of mineral sites"
):
    tmp = get_mineral_site_data(batch_ids)
    assert len(tmp) == len(batch_ids), (len(tmp), len(batch_ids))
    id2siteinfo.update(tmp)

retrieving additional information of mineral sites: 100%|██████████| 42/42 [00:02<00:00, 16.81it/s]


In [12]:
grade_tonnage_resp.json()[0]

{'ms': 'https://minmod.isi.edu/resource/site__api-cdr-land-v1-docs-documents__02b1591cfad6809d1d9ea67ea04b34d2df8b16d5f34e4bba701a7924bab9c3b48d',
 'ms_name': 'NI 43-101 Technical Report for the Läntinen Koillismaa project in Finland dated May, 2021.pdf',
 'country': 'Finland',
 'state_or_province': 'Central Finland',
 'loc_crs': 'EPSG:4326',
 'loc_wkt': 'POINT(28.128333 65.905722)',
 'doc_url': 'https://api.cdr.land/v1/docs/documents/02b1591cfad6809d1d9ea67ea04b34d2df8b16d5f34e4bba701a7924bab9c3b48d',
 'tot_contained_metal': 7.6917e-05,
 'total_tonnage': 130.9,
 'total_grade': 5.876012223071046e-05}

In [37]:
len(site_ids)

4105

In [18]:
r = httpx.get(
    "https://minmod.isi.edu/api/v1/mineral_site_deposit_types/nickel", verify=False
)

In [27]:
r = httpx.get(
    "https://minmod.isi.edu/api/v1/mineral_site_deposit_types/zinc",
    verify=False,
    timeout=None,
)

In [26]:
grade_tonnage_resp

NameError: name 'grade_tonnage_resp' is not defined

In [25]:
len(r.json())

28658

In [7]:
r = httpx.get(
    "https://api.cdr.land/v1/minerals/sites?fields=",
    headers=headers,
    follow_redirects=True,
)

In [9]:
dat = r.json()

In [15]:
dat[0]

{'system_version': '0.1.0',
 'name': 'Alma',
 'source_id': 'https://mrdata.usgs.gov/mrds',
 'site_type': 'Past Producer',
 'id': 'site__mrdata-usgs-gov-mrds__10036253',
 'system': 'bulkingest',
 'validated': False,
 'record_id': '10036253',
 'site_rank': 'B',
 'country': [{'name': 'United States',
   'id': '75fd64387e1749b18630163215b6f22d',
   'mineral_site_id': 'site__mrdata-usgs-gov-mrds__10036253'}],
 'mineral_inventory': [{'ore_value': None,
   'material_form_conversion': None,
   'mineral_site_id': 'site__mrdata-usgs-gov-mrds__10036253',
   'ore_unit': '',
   'material_form_observed_name': '',
   'cutoff_grade': None,
   'id': '01a9903730d1483dae258b78f07a3d64',
   'contained_metal': None,
   'cutoff_unit': '',
   'date': '',
   'grade_value': None,
   'grade_unit': '',
   'zone': '',
   'commodity': 'Sulfur',
   'material_form': None,
   'commodity_observed_name': 'Sulfur',
   'material_form_unit': '',
   'documents': [],
   'records': [{'mineral_inventory_id': '01a9903730d1483d

In [13]:
len(dat)

10